In [4]:
#Daten in Trainingsdaten und Validierungsdaten aufteilen
import json

f = open('/Users/wanjagerber/downloads/veniceAnnotations.json')
data = json.load(f)
TAGGED_DATA = data['annotations']

print(len(TAGGED_DATA)*0.8)

import random 

random.shuffle(TAGGED_DATA)
train_data = TAGGED_DATA[:76]
val_data = TAGGED_DATA[76:]

print("Trainingsdaten: " + str(len(train_data)))
print("Validierungsdaten: " + str(len(val_data)))


with open ('/Users/wanjagerber/desktop/burneyNER/trainData.json', 'w', encoding='utf-8') as train: 
    json.dump(train_data, train, ensure_ascii=False, indent=4)
with open ('/Users/wanjagerber/desktop/burneyNER/valuationData.json', 'w', encoding='utf-8') as val: 
    json.dump(val_data, val, ensure_ascii=False, indent=4)


76.0
Trainingsdaten: 76
Validierungsdaten: 19



Hier muss nun folgender Zusatz in die Trainings- und Validierungsdaten eingefügt werden, sowie eine ```python
'}'
``` um das Dokument zu schliessen. Ausserdem existiert noch immer das Prolem der 'null'-Werte, die vom NER-Annotator generiert werden. Durch die Formatierung konnten sie reduziert, aber nicht vollständig umgangen werden. Daher müssen die restlichen 'Null'-Werte hier durch Suchen und Ersetzen entfernt werden.


In [ ]:
{"classes": ["DATE","LOCATION", "PERSON","INSTRUMENT","MUSIC"],
    "annotations":



Dann können die Dateien ins spacy-Format konvertiert werden.


In [9]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json

#neues spacy Model laden
nlp = spacy.load("en_core_web_md") 

#DocBin Objekt erstellen
db = DocBin() 

#Trainingsdaten
f = open('/Users/wanjagerber/desktop/burneyNER/trainData.json')
TRAIN_DATA = json.load(f)

for text, annot in tqdm(TRAIN_DATA['annotations']): 
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents 
    db.add(doc)

#Docbin Objekt speichern
db.to_disk("/Users/wanjagerber/desktop/burneyNER/trainData.spacy") 

100%|██████████| 67/67 [00:00<00:00, 139.08it/s]


In [10]:
#Valuiierungsdaten
f = open('/Users/wanjagerber/desktop/burneyNER/valuationData.json')
VAL_DATA = json.load(f)

for text, annot in tqdm(VAL_DATA['annotations']): 
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents 
    db.add(doc)

#Docbin Objekt speichern
db.to_disk("/Users/wanjagerber/desktop/burneyNER/valuationData.spacy") 

100%|██████████| 18/18 [00:00<00:00, 354.64it/s]


In [12]:
#Testdata
f = open('/Users/wanjagerber/desktop/burneyNER/testData.json')
TEST_DATA = json.load(f)

for text, annot in tqdm(TEST_DATA['annotations']): 
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents 
    db.add(doc)

#Docbin Objekt speichern
db.to_disk("/Users/wanjagerber/desktop/burneyNER/testData.spacy")

100%|██████████| 11/11 [00:00<00:00, 794.47it/s]
